In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
from imdb import IMDb

In [2]:
# Get data frame with data from movies
df = pd.read_csv('western.csv', sep=';')

In [3]:
df.head(10)

,Unnamed: 0,title,plot
0,0,Wild Bill,The early career of legendary lawman Wild Bill...
1,1,Wyatt Earp,Wyatt Earp is a movie about a man and his fami...
2,2,Bad Girls,Four prostitutes join together to travel the O...
3,3,The Outlaw,"Western legends Pat Garrett, Doc Holliday and ..."
4,4,The Shooter,The story is of a small town in the early west...
5,5,Terror in a Texas Town,A Swedish whaler is out for revenge when he fi...
6,6,Pale Rider,A mysterious preacher protects a humble prospe...
7,7,Alvarez Kelly,"During the Civil War, Mexican cattleman Alvare..."
8,8,High Plains Drifter,A gunfighting stranger comes to the small sett...
9,9,Rough Night in Jericho,A former deputy and a strong-willed widow dete...


In [4]:
es = Elasticsearch()

In [5]:
es.indices.create(index="movie-index", ignore=400)

{u'acknowledged': True, u'shards_acknowledged': True}

In [6]:
for i in range(1, len(df.index)):
    es.index(index='movie-index', doc_type='movie', body={
        'id': i,
        'title': df.loc[i]['title'],
        'plot': df.loc[i]['plot']
    })

In [7]:
res = es.search(index='movie-index', body={"query": {"fuzzy" : { "title" : "texs"}}}, size=3)

In [27]:
res['hits']['hits']

[{u'_id': u'AVyn78M6E-HKshLZMgA2',
  u'_index': u'movie-index',
  u'_score': 1.8507599,
  u'_source': {u'id': 5,
   u'plot': u'A Swedish whaler is out for revenge when he finds out that a greedy oil man murdered his father for their land.',
   u'title': u'Terror in a Texas Town'},
  u'_type': u'movie'}]

In [31]:
for doc in res['hits']['hits']:
    print doc['_source']['plot']

A Swedish whaler is out for revenge when he finds out that a greedy oil man murdered his father for their land.


In [33]:
# Delete index when too many values are indexed
#es.indices.delete(index='movie-index', ignore=[400, 404])
#es.indices.delete(index='western', ignore=[400, 404])

{u'acknowledged': True}